## Hands-on session 2.1 - working with kinetic simulations: Epperlein-Short test

This session covers basic electron kinetics features in ReMKiT1D. It is not meant to guide the reader through detailed model construction, and most of the model is prebuilt. 

A companion file `es_models.py` is supplied that creates the necessary kinetic models/terms using `common_models`. 

The reader is required to complete the setup of the grid and the variables.

Demonstrated concepts:

- Setting up a full ReMKiT1D grid (x,h,v)
- Parallelization in harmonics
- Distribution variables
- Moments and related derivations
- More prebuilt derivations
- Inspecting kinetic data 

In [ ]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds

from es_models import addESModels,kappa0
import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt


### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for writing the files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/day_2_1/"
rk.setHDF5Path(hdf5Filepath) 

### MPI

In kinetic simulations, we have the additional option to add processors in the harmonic direction. If you use 4 or more harmonics (`lMax>2`) try increasing numProcsH (keep in mind that the total number of harmonics should be divisible by numProcsH!)

The total number of MPI processes ReMKiT1D will expect then will be `numProcsX*numProcsH`.

In [ ]:
rk.setMPIData(numProcsX=4,numProcsH=1) # use numProcsH to change the number of harmonic direction processes

### Normalization

Let's set a higher normalization temperature than usual

In [ ]:
rk.setNormDensity(1.0e19) #n_0
rk.setNormTemperature(100.0) #T_0
rk.setNormRefZ(1.0) # reference ion charge for e-i collision time

### Grid initialization

We now initialize the full grid, where we need to specify the spatial and velocity grids, as well as the number of distribution function harmonics

In [ ]:
dx = 150.0 # Cell width in reference e-i collision mfps (use this to control the perturbation wavelength in the ES test - larger numbers are more collisional) 

xGrid = dx*np.ones(64)

# Velocity grid setup 
vGrid = [0.0307]
for i in range(1,80):
    vGrid.append(vGrid[i-1]*1.025)
vGrid = np.array(vGrid)

lMax = 1 # Highest used l harmonic (the total number of harmonics is lMax+1, including l=0)

gridObj = Grid(xGrid, vGrid, lMax, interpretXGridAsWidths=True, interpretVGridAsWidths=True, isPeriodic=True)
L = sum(xGrid) # Length of the spatial grid

rk.grid = gridObj

### Variables

We will need the following variables for the Epperlein-Short test:

- The electron distribution function variable
- The density and the temperature of our electrons 
- An electric field variable

Let's start by setting the initial values of our variables. 

**Note**: Here we use the fact that the distribution function is normalized to $n_0/v_{th}^3$ by default, and that $eT_0=m_ev_{th}^2/2$.

In [ ]:
n = np.ones(gridObj.numX())
T = 1.0 + 0.001*np.sin(2*np.pi*gridObj.xGrid/L) # A small temperature perturbation around the reference value T_0=100eV

# A Maxwellian with the above n and T for the l=0 harmonic and 0 for all the others
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = np.pi**(-1.5) * T[i] ** (-1.5) * n[i] * np.exp(-gridObj.vGrid**2/T[i])

##### Some numerical considerations

We have calculated the discretized values of our analytical Maxwellian on the velocity grid we supplied. Let's check whether the numerical integration in ReMKiT1D will have the correct density. We can do this using the `velocityMoment` function of the `Grid` class.

In [ ]:
numerical_dens = gridObj.velocityMoment(distFun=f,momentOrder=0,momentHarmonic=1) # Note, we use the Fortran indexing here (the first harmonic is the l=0 harmonic)
numerical_dens[0]

As you can see, and might expect, this is not equal to the density we requested. Some discretization errors can be corrected easily by rescaling the distribution function:

In [ ]:
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = n[i] *f[i,gridObj.getH(0)-1,:]/numerical_dens[i]

We can now confirm that the numerical integration will give us the correct density

In [ ]:
gridObj.velocityMoment(distFun=f,momentOrder=0,momentHarmonic=1)[0]

We can now proceed to add the distribution function to the wrapper

In [ ]:
rk.addVarAndDual("f",f,isDistribution=True,isCommunicated=True)

**NOTE**: Distribution variables on staggered grids are treated the following way:

- The even harmonics of "f" live on cell centers, and the odd live on cell edges
- The opposite is true for "f_dual" - even harmonics are interpolated onto cell edges, odd harmonics are interpolated into cell centers  

We also need to use a number of [built-in derivations](https://remkit1d-python.readthedocs.io/en/latest/custom_fluid.html#Textbook-objects-and-derivations). In particular we want to be able do derive the electron temperature. For this we pass the electron species index (more on Species in the next hands-on session) to the setup of the standard textbook object:

In [ ]:
rk.setStandardTextbookOptions(tempDerivSpeciesIDs=[0]) # This will enable us to use the temperature derivation for electrons directly

We'll use the built-in moment derivations to get the density and energy density of the electrons

In [ ]:
rk.addVarAndDual("n",n,units='$10^{19} m^{-3}$',isDerived=True,derivationRule=sc.derivationRule("densityMoment",["f"]))
rk.addVar("W",isDerived=True,derivationRule=sc.derivationRule("energyMoment",["f"])) # we only need the energy density at cell centers to get the temperature

The temperature derivation for the electrons will be generated with the name `tempFromEnergye`, and will require three variables passed to it, the energy density, particle density, as well as a particle flux. Since our problem setup has no flows, we can use a dummy variable instead of the flux.

In [ ]:
rk.addVar("zeroVar",isDerived=True,outputVar=False) # We can suppress the outputting of a variable using the outputVar flag
rk.addVarAndDual("T",T,units='$100eV$',isDerived=True,derivationRule=sc.derivationRule("tempFromEnergye",["W","n","zeroVar"]),isCommunicated=True)

Finally, we add the electric field and time variables

In [ ]:
rk.addVarAndDual("E",primaryOnDualGrid=True,isCommunicated=True)
rk.addVar("time",isScalar=True,isDerived=True)

### Adding the required models

The `addESModels` function can be used to add all the relevant terms and models for this example

In [ ]:
addESModels(wrapper=rk,
            lMax=lMax,
            distFunName="f",
            eFieldName="E",
            elTempName="T",
            elDensName="n");

### Adding heat flux diagnostics

We want to be able to inspect the deviation of the heat flux obtained from the electron distribution function with that predicted by the classical Braginskii model. 

Let's first add a variable for the heat flux using one of the pre-built moment derivations 

In [ ]:
rk.addVarAndDual("q",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("heatFluxMoment",["f"]))

Let's get ReMKiT1D to calculate the Braginskii heat flux at each step. We will need the Coulomb logarithm, the conductivity, and the temperature gradient. Let's use a combination of built-in derivations and tree derivations to calculate this.

First, we need the Coulomb logarithm. An e-i Coulomb logarithm derivation is added for each ion species in a ReMKiT1D simulation. The `addESModels` function adds a species named `D+`, so we use the derivation name `logLeiD+`. 

In [ ]:
rk.addVar("logLei",isDerived=True,derivationRule=sc.derivationRule("logLeiD+",["T","n"]))

The conductivity $\kappa$ will be proportional to $T^{5/2}/\text{logL}$. For convenience, the constant of proportionality can be obtained using the `kappa0` function from `es_models.py`. We can use this to add a variable to calculate the heat Braginskii conductivity

In [ ]:
rk.addVar("kappa",isDerived=True,derivationRule=sc.derivationRule("kappa",["T","logLei"]),derivOptions=treeDerivation(kappa0(rk)*Node("T")**2.5/Node("logLei")))

To construct the heatflux $q=-\kappa \nabla T$, we need the temperature gradient. One way of doing this is using the built-in gradient derivation `gradDeriv`, passing it the variable we want to calculate the gradient of, which should live on the regular grid.

In [ ]:
rk.addVar("gradT",isDerived=True,derivationRule=sc.derivationRule("gradDeriv",["T"]))

Finally we can assemble the heat flux using the two added variables `kappa` and `gradT` using the calculation tree approach.

In [ ]:
rk.addVar("qT",isDerived=True,derivationRule=sc.derivationRule("qT",["kappa","gradT"]),derivOptions=treeDerivation(-Node("kappa")*Node("gradT")))

### Setting up the time integration options

Below is a standard setup for the Epperlein-Short test time integration with ReMKiT1D.

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["f"])

rk.addIntegrator("BE", integrator)

rk.setIntegratorGlobalData(initialTimestep=0.05)

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1", bdeStep.dict())

Nt = 300
rk.setFixedNumTimesteps(Nt)
rk.setFixedStepOutput(Nt/30)

rk.setPETScOptions(cliOpts="-pc_type bjacobi -sub_pc_factor_shift_type nonzero",kspSolverType="gmres")

### Create config 

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=30
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames, varsToIgnore=["zeroVar"]) # Ignore the variables that aren't in the output
loadedData


We can inspect fluid variables using the standard dashboard tool

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show()

A way to explore distribution variables is available using the `distDynMap` function

In [ ]:
dashboard.distDynMap().show()

Finally, we can compare the simulated and classical values of the heat flux

In [ ]:
dashboard.fluidMultiComparison(["q","qT"])